In [1]:
import sys
import os
sys.path.append(os.path.abspath("../"))

from src.utils.utils import *


In [2]:
data = load_csv("data/benin_raw.csv")


NameError: name 'load_csv' is not defined

In [ ]:
def load_dataset(path):
    import pandas as pd
    return pd.read_csv(path)
